## Transfer Learning and Fine Tuning

### Loading the dataset

In [1]:
 # installs the datasets package and imports the load_dataset function.

!pip install datasets

from datasets import load_dataset

# Load a subset of the SQuAD dataset for demonstration (first 5000 examples)
squad = load_dataset("squad", split="train[:5000]")

# This part loads a subset of the SQuAD dataset containing the first 5000 examples and prints the dataset.

print(squad)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 5000
})


In [2]:
squad = squad.train_test_split(test_size=0.2)
squad["train"][0]

{'id': '56cd5ccc62d2951400fa6533',
 'title': 'Sino-Tibetan_relations_during_the_Ming_dynasty',
 'context': 'However, Lok-Ham Chan, a professor of history at the University of Washington, writes that Changchub Gyaltsen\'s aims were to recreate the old Tibetan Kingdom that existed during the Chinese Tang dynasty, to build "nationalist sentiment" amongst Tibetans, and to "remove all traces of Mongol suzerainty." Georges Dreyfus, a professor of religion at Williams College, writes that it was Changchub Gyaltsen who adopted the old administrative system of Songtsän Gampo (c. 605–649)—the first leader of the Tibetan Empire to establish Tibet as a strong power—by reinstating its legal code of punishments and administrative units. For example, instead of the 13 governorships established by the Mongol Sakya viceroy, Changchub Gyaltsen divided Central Tibet into districts (dzong) with district heads (dzong dpon) who had to conform to old rituals and wear clothing styles of old Imperial Tibet. Va

### Loading DistilBERT

In [3]:
# using the transformers library to load a pre-trained tokenizer.
#The AutoTokenizer class allows you to automatically load the appropriate tokenizer for a given pre-trained model.
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

### Preprocessing the data

Tokenization: It tokenizes the questions and context by using the provided tokenizer. Tokenization is the process of splitting text into individual tokens (words, subwords, or characters) that can be processed by a model. The tokens are limited to a maximum length of 384, and they are truncated based on the context.

Offset mappings provide a way to connect the tokenized input to the original words in the context. This is useful for determining the exact positions of words in the original context after tokenization.

If the answer is not entirely contained within the context, start and end positions are designated as zero. Otherwise, the token positions corresponding to the start and end of the answer within the context are identified.

In [4]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [5]:
# The preprocess_function is applied to the dataset using the map method, to tokenize and preprocess the dataset.

tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

 The `DefaultDataCollator` is used to evaluation of machine learning models, especially in the context of natural language processing tasks. It helps in converting the raw input data into the format expected by the model, handling tasks like padding sequences to the same length and converting them into tensors for efficient processing by the machine learning framework, such as TensorFlow.  

In [6]:
from transformers import DefaultDataCollator
data_collator = DefaultDataCollator(return_tensors="tf")

### Fine Tuning

In [7]:
from transformers import create_optimizer
batch_size = 16
num_epochs = 3
total_train_steps = (len(tokenized_squad["train"]) // batch_size) * num_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=total_train_steps,
)

TFAutoModelForQuestionAnswering is a class from the Hugging Face Transformers library that allows you to easily create a TensorFlow-based model specifically designed for answering questions based on given contexts

In [8]:
from transformers import TFAutoModelForQuestionAnswering
model = TFAutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForQuestionAnswering: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it

In [9]:
model.summary()

Model: "tf_distil_bert_for_question_answering"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 qa_outputs (Dense)          multiple                  1538      
                                                                 
 dropout_19 (Dropout)        multiple                  0 (unused)
                                                                 
Total params: 66364418 (253.16 MB)
Trainable params: 66364418 (253.16 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_squad["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_squad["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [11]:
model.compile(optimizer=optimizer)

In [12]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3)

Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
250/250 [==============================] - 11803s 47s/step - loss: 3.4470 - val_loss: 2.0087
Epoch 2/3
250/250 [==============================] - 11698s 47s/step - loss: 1.6752 - val_loss: 1.6517
Epoch 3/3
250/250 [==============================] - 11642s 47s/step - loss: 1.2402 - val_loss: 1.5840


### Model Testing

In [13]:
import tensorflow as tf
question = "What is the name of the town where Chhota Bheem and his friends live?"
prompt_for_context = "Chhota Bheem and his friends live in the town of Dholakpur, where they have many adventures and fun-filled experiences."
inputs = tokenizer(question, prompt_for_context, return_tensors="tf")
outputs = model(**inputs)
answer_start_index = int(tf.math.argmax(outputs.start_logits, axis=-1)[0])
answer_end_index = int(tf.math.argmax(outputs.end_logits, axis=-1)[0])
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)

'dholakpur'

Transfer learning is a machine learning technique where a model trained on one task is repurposed or fine-tuned for another related task. It's a powerful approach that leverages knowledge gained from solving one problem and applies it to a different but related problem, often leading to improved performance, faster training, and reduced data requirements.